In [ ]:
import pandas as pd


df = pd.read_csv("../data/processed_lead_data.csv")
df.head()

,request_id,time_on_page,actions,device,timezone,region,domain,is_spam,name_length,name_has_test,...,landing_page_Скидки на Audi,browser_Edge,browser_Firefox,browser_Internet Explorer,browser_Opera,browser_Safari,email_domain_inbox.ru,email_domain_mail.ru,email_domain_unknown,email_domain_yandex.ru
0,1,3.420066,5.0,mobile,Europe/Helsinki,Новосибирск,rolf-cars.ru,1,11,0,...,False,False,True,False,False,False,True,False,False,False
1,2,6.150441,4.0,mobile,Europe/Helsinki,Казань,rolf-sale.ru,1,15,0,...,False,False,False,False,False,False,False,True,False,False
2,3,31.295270,8.0,mobile,Asia/Krasnoyarsk,Новосибирск,rolf-sale.ru,0,13,0,...,True,False,False,False,False,False,False,False,False,True
3,4,1.294227,2.0,desktop,Europe/Moscow,Санкт-Петербург,rolf-service.ru,0,12,0,...,False,False,False,False,False,False,False,True,False,False
4,5,56.006976,4.0,desktop,Europe/Moscow,Екатеринбург,rolf-sale.ru,0,13,0,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
import pandas as pd
import os
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import precision_score, recall_score
import pickle

# Путь к данным
data_dir = "../data"
input_path = os.path.join(data_dir, "processed_lead_data.csv")
output_model_path = os.path.join(data_dir, "spam_model.pkl")

# Загрузка данных
df = pd.read_csv(input_path)

# Разделение на признаки и целевую переменную
X = df.drop(columns=["is_spam"])
y = df["is_spam"]

# Определение задачи (бинарная классификация)
task = Task("binary")

# Настройка LightAutoML
automl = TabularAutoML(
    task=task,
    timeout=600,  # 10 минут на обучение
    cpu_limit=4,  # Ограничение по количеству ядер
    general_params={"use_algos": [["lgb", "cb", "xgb"]]},  # Используемые алгоритмы
    selection_params={"mode": 0},  # Отключение feature selection для скорости
    tuning_params={"max_tuning_iter": 5}  # Ограничение итераций тюнинга
)

# Обучение модели
oof_pred = automl.fit_predict(
    df,
    roles={"target": "is_spam", "drop": []},
    verbose=1
)

# Предсказания на обучающей выборке
y_pred = (oof_pred.data[:, 0] > 0.5).astype(int)

# Оценка метрик
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Сохранение модели
with open(output_model_path, "wb") as f:
    pickle.dump(automl, f)
print(f"Модель сохранена в {output_model_path}")

[13:31:15] Stdout logging level is INFO.
[13:31:15] Task: binary

[13:31:15] Start automl preset with listed constraints:
[13:31:15] - time: 600.00 seconds
[13:31:15] - CPU: 4 cores
[13:31:15] - memory: 16 GB

[13:31:15] Train data shape: (100000, 30)

[13:31:18] Layer 1 train process start. Time left 596.52 secs
[13:31:21] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[13:31:26] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = 0.9867878445345245
[13:31:26] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[13:31:26] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoost ...
[13:31:31] Fitting Lvl_0_Pipe_0_Mod_1_CatBoost finished. score = 0.9959322018958365
[13:31:31] Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
[13:31:31] Start fitting Lvl_0_Pipe_0_Mod_2_XGBoost ...
[13:31:33] Fitting Lvl_0_Pipe_0_Mod_2_XGBoost finished. score = 0.9964102095336328
[13:31:33] Lvl_0_Pipe_0_Mod_2_XGBoost fitting and predicting completed
[13:31:33] Time left 582.28 secs

[13:31:33] Laye